In [ ]:
%pip install gherkin-official

In [131]:
from gherkin.parser import Parser
from typing import List

class Scenario:
    def __init__(self, name: str) -> None:
        self.name = name
        self.context: List[str] = []
        self.steps: List[str] = []
        self.expect: List[str] = []
    
    def __str__(self) -> str:
        return f"Context: {self.context}\nSteps: {self.steps}\nExpect: {self.expect}"
    
    def __repr__(self) -> str:
        return f"Scenario({str(self)})"

def read_scenarios(feature_file_path: str) -> List[Scenario]:
    scenarios: List[Scenario] = []
    with open(feature_file_path, 'r', encoding='utf8') as file:
        feature_file_content = file.read()
        parser = Parser()
        parsed_feature = parser.parse(feature_file_content)
        parsed_scenarios = parsed_feature["feature"]["children"]

        for parsed_scenario in parsed_scenarios:
            scenario = Scenario(parsed_scenario["scenario"]["name"])
            scenarios.append(scenario)
            last_keywork: str = None

            for step in parsed_scenario["scenario"]["steps"]:
                
                keyword = step["keywordType"]
                if keyword == "Conjunction":
                    keyword = last_keywork
                else:
                    last_keywork = keyword
                
                if keyword == 'Context':
                    scenario.context.append(step['text'])
                elif keyword == 'Action':
                    scenario.steps.append(step['text'])
                elif keyword == 'Outcome':
                    scenario.expect.append(step['text'])
                else:
                    print("Parser missing", step)

    return scenarios


In [136]:
from lavague.core import WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium.base import SeleniumDriver

url = "https://www.laposte.fr/"
feature_file = "demo_laposte.feature"
scenarios = read_scenarios(feature_file)
scenario = scenarios[0]

driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(driver)
agent = WebAgent(
    world_model,
    action_engine,
)
agent.get(url)
agent.prepare_run()

for step in scenario.steps:
    agent.run_step(step)

# We run the asserts, the agent should COMPLETE
scenario_completion = agent.run_step(" and ".join(scenario.expect))
if scenario_completion:
    print("Scenario completed successfully", scenario_completion.output)
else:
    print("Scenario might not be completed")

logs = agent.logger.return_pandas()

2024-07-31 14:42:40,328 - INFO - Screenshot folder cleared
2024-07-31 14:42:51,039 - INFO - Thoughts:
- The current screenshot shows a webpage from La Poste with a cookie consent banner at the bottom.
- The objective is to click on "J'accepte" to accept cookies.
- The "J'accepte" button is visible and likely interactable in the cookie consent banner.
- The next step should involve clicking on the "J'accepte" button to accept cookies.

Next engine: Navigation Engine
Instruction: Click on the "J'accepte" button in the cookie consent banner.
2024-07-31 14:43:04,463 - INFO - Thoughts:
- The current screenshot shows the homepage of La Poste, a postal service website.
- The objective is to click on "Envoyer un colis".
- The "Envoyer un colis" button is visible on the page.
- The next step should involve clicking on the "Envoyer un colis" button to proceed with the objective.

Next engine: Navigation Engine
Instruction: Click on the "Envoyer un colis" button.
2024-07-31 14:43:23,858 - INFO - 

Scenario completed successfully The cost is "34,70 €".


In [149]:
# Convert pandas actions logs to pytest code

import yaml
import re

pytest_code = f"""
import pytest
from pytest_bdd import scenarios, given, when, then
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Constants
BASE_URL = '{url}'

# Scenarios
scenarios('{feature_file}')

# Fixtures
@pytest.fixture
def browser():
    driver = WebDriver()
    driver.implicitly_wait(10)
    yield driver
    driver.quit()

# Steps
"""

indent = "    "
indent_pass = indent + "pass"

def to_snake_case(s: str):
    s = s.lower()
    s = re.sub(r'[^\w\s]', '_', s)
    s = re.sub(r'\s+', '_', s)
    s = re.sub(r'__+', '_', s)
    return s.strip('_')

def get_click_action(xpath: str):
    return (
f"""element = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '{xpath}'))
    )
    browser.execute_script('arguments[0].click();', element)
"""
    )

def get_set_value_action(xpath, value, enter=False):
    code = get_click_action(xpath)
    code += indent + f"element.clear()"
    code += "\n" + indent + f"element.send_keys('{value}')"
    if enter:
        code += "\n" + indent + f"element.send_keys('\ue007')"
    return code

def get_select_action(xpath, value):
    return (
f"""element = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '{xpath}'))
    )
    select = Select(element)
    try:
        select.select_by_value('{value}')
    except:
        select.select_by_visible_text('{value}')
"""
    )

def get_nav_action_code(action):
    action_name = action["name"]
    args = action["args"]
    xpath = None if args is None else args.get("xpath")

    if action_name == "click":
        return get_click_action(xpath)
    elif action_name == "setValue":
        return get_set_value_action(xpath, args["value"])
    elif action_name == "setValueAndEnter":
        return get_set_value_action(xpath, args["value"], True)
    elif action_name == "dropdownSelect":
        return get_select_action(xpath, args["value"])
    
    return "pass"

def get_nav_control_code(instruction):
    if "SCROLL_DOWN" in instruction:
        return "browser.scroll_down()"
    if "SCROLL_UP" in instruction:
        return "browser.scroll_up()"
    if "WAIT" in instruction:
        return "import time\n    time.sleep(5)"
    if "BACK" in instruction:
        return "browser.back()"
    if "SCAN" in instruction:
        return "pass"
    if "MAXIMIZE_WINDOW" in instruction:
        return "browser.maximize_window()"
    if "SWITCH_TAB" in instruction:
        tab_id = int(instruction.split(" ")[1])
        return f"browser.switch_tab(tab_id={tab_id})"

def get_pytest_when(gherkin_step: str, engine: str, engine_log: str, instruction: str) -> str:
    step = gherkin_step.replace("'", "\\'")
    method_name = to_snake_case(gherkin_step)
    if engine == "Navigation Engine":
        actions = yaml.safe_load(engine_log)[0]["actions"]
        actions_code = "\n".join([
            indent + get_nav_action_code(a["action"]) for a in actions
        ]) or indent_pass
    elif engine == "Navigation Controls":
        actions_code = indent + get_nav_control_code(instruction)
    else:
        actions_code = indent_pass
    return f"""
@when('{step}')
def {method_name}(browser: WebDriver):
{actions_code}
"""

# GENERATE @given
is_first = True
for setup in scenario.context:
    step = setup.replace("'", "\\'")
    method_name = to_snake_case(setup)
    if is_first:
        is_first = False
        code = "browser.get(BASE_URL)"
    else:
        code = "pass"
    pytest_code += f"""
@given('{step}')
def {method_name}(browser: WebDriver):
    {code}
"""

# GENERATE @when
for index, row in logs.iterrows():
    if index < len(scenario.steps):
        gherkin_step = scenario.steps[index]
        pytest_code += get_pytest_when(gherkin_step, row["engine"], row["code"], row["instruction"])

# GENERATE @then
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from lavague.core.retrievers import SemanticRetriever
from llama_index.core import QueryBundle

llm = OpenAI(model="gpt-4o")
embedding = OpenAIEmbedding(model="text-embedding-3-small")
retriever = SemanticRetriever(embedding=embedding, xpathed_only=False)

def generate_assert_code(expect: str):
    html = retriever.retrieve(QueryBundle(expect), [driver.get_html()])
    prompt = f"""
You are an expert in software testing frameworks and Python code generation. You answer in python markdown only and nothing else.
Your only goal is to use the provided Gherkin and HTML nodes to generate a valid pytest-bdd python assert statement. 
- Include all necessary imports and fixtures.
- If element selection is needed, prefer XPath based on class or text content to fetch it. 
- You answer in python code only and nothing else.
- You have access to a browser variable of type selenium.webdriver.chrome.webdriver.Chrome

```python
browser: WebDriver
# assert code here
```

I will provide an example below:
----------
Gherkin: Then the cost should be "34,70 €"
HTML: <div><span>Cost:</span><span class="calculator__cta__price">34,70 €</span></div>
                
Resulting pytest code: 
cost_element = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.XPATH, "//span[@class='calculator__cta__price']"))
)
actual_cost = cost_element.text.strip()
assert actual_cost == expected_cost

----------
Given this information, generate a valid pytest-bdd assert instruction with the following inputs:
Gherkin expect of the feature to be tested: Then {expect}\n
Potentially relevant HTML that you may use to help you generate the assert code: {html}\n
"""
    code = llm.complete(prompt).text
    code = code.replace("```python", "").replace("```", "")
    code = code.replace("# assert code here", "")
    return "\n".join([indent + l for l in code.splitlines()])

for expect in scenario.expect:
    step = expect.replace("'", "\\'")
    method_name = to_snake_case(expect)
    assert_code = generate_assert_code(expect)
    pytest_code += f"""
@then('{step}')
def {method_name}(browser: WebDriver):
{assert_code}
"""

print(pytest_code)

with open("test_file.py", "w") as file:
    file.write(pytest_code)


import pytest
from pytest_bdd import scenarios, given, when, then
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# Constants
BASE_URL = 'https://www.laposte.fr/'

# Scenarios
scenarios('demo_laposte.feature')

# Fixtures
@pytest.fixture
def browser():
    driver = WebDriver()
    driver.implicitly_wait(10)
    yield driver
    driver.quit()

# Steps

@given('I am on the homepage')
def i_am_on_the_homepage(browser: WebDriver):
    browser.get(BASE_URL)

@when('I click on "J\'accepte" to accept cookies')
def i_click_on_j_accepte_to_accept_cookies(browser: WebDriver):
    element = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div/div[2]/div[2]/button'))
    )
    browser.execute_script('arguments[0].click();', element)


@when('I click on "Envoyer un colis"')


In [150]:
!pytest test_file.py

13716.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.2, pluggy-1.5.0
rootdir: /home/adeprez/workspace/LaVague
configfile: pyproject.toml
plugins: anyio-4.3.0, bdd-7.2.0
collected 1 item                                                               

test_file.py .                                                           [100%]

============================== 1 passed in 23.14s ==============================
